In [1]:
import graphlab
import time

In [2]:
start_time = time.time()
song_data = graphlab.SFrame('../data/song_data.gl/')
print("--- time to load data =  %s seconds ---" % (time.time() - start_time))
print type(song_data), song_data.shape


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: Z:\WINTEMP\graphlab_server_1479012159.log.0


This non-commercial license of GraphLab Create for academic use is assigned to bmatthewtaylor@gmail.com and will expire on October 27, 2017.
--- time to load data =  1.75100016594 seconds ---
<class 'graphlab.data_structures.sframe.SFrame'> (1116609, 6)


In [3]:
print song_data.column_names()

['user_id', 'song_id', 'listen_count', 'title', 'artist', 'song']


Q1. Which of the artists below have had the __most__ unique users listening to their songs?
Calcs below.

In [4]:
kayne = song_data.filter_by('Kanye West', 'artist')
print "kayne.shape = ", kayne.shape
#print kayne.head(5)
print "No of unique listeners = ", len(set(kayne['user_id']))

kayne.shape =  (3775, 6)
No of unique listeners =  2522


In [5]:
fooFighters = song_data.filter_by('Foo Fighters', 'artist')
print "fooFighters.shape = ", fooFighters.shape
print "No of unique listeners = ", len(set(fooFighters['user_id']))

fooFighters.shape =  (3429, 6)
No of unique listeners =  2055


In [6]:
taylorSwift = song_data.filter_by('Taylor Swift', 'artist')
print "taylorSwift.shape = ", taylorSwift.shape
print "No of unique listeners = ", len(set(taylorSwift['user_id']))

taylorSwift.shape =  (6227, 6)
No of unique listeners =  3246


In [7]:
ladyGaGa = song_data.filter_by('Lady GaGa', 'artist')
print "ladyGaGa.shape = ", ladyGaGa.shape
print "No of unique listeners = ", len(set(ladyGaGa['user_id']))

ladyGaGa.shape =  (4129, 6)
No of unique listeners =  2928


Q1: answer = 'Taylor Swift' has the most unique listener.

Q2. Which of the artists below is the most popular artist, the one with highest total listen_count, in the data set?

In [8]:
song_data.filter_by('Kings of Leon', 'artist').shape
#NB 'Kings of Leon' did not have any rows in song_data

(0, 6)

In [9]:
print song_data.filter_by('Taylor Swift', 'artist')['listen_count'].sum()
print song_data.filter_by('Kings of Leon', 'artist')['listen_count'].sum()
print song_data.filter_by('Coldplay', 'artist')['listen_count'].sum()
print song_data.filter_by('Lady GaGa', 'artist')['listen_count'].sum()

19376
0
35362
12224


Q2 : Answer = Coldplay with 35362 total listens.

Q3. Which of the artists below is the least popular artist, the one with smallest total listen_count, in the data set?

In [10]:
print song_data.filter_by('William Tabbert', 'artist')['listen_count'].sum()
print song_data.filter_by('Velvet Underground & Nico', 'artist')['listen_count'].sum()
print song_data.filter_by('Kanye West', 'artist')['listen_count'].sum()
print song_data.filter_by('The Cool Kids', 'artist')['listen_count'].sum()

14
80
9992
73


Q3: Answer : 'William Tabbert' with 14 users.
Q4 : Using the first 10,000 unique users only in the test data, use the personalized_model learned on the training data to recommend 1 song to each user. What’s the most recommended song?

steps
1. create list of first 10,000 unique users from the test data.
2. subset the test_data using this list of the first 10,000 unique users
3. reccommend 1 song for each user, using the personalized_model.
4. identify the most reccomended song.

In [11]:
train_data,test_data = song_data.random_split(.8,seed=0)
#NB: seed=0 makes this repeatable.
print type(train_data), train_data.shape

<class 'graphlab.data_structures.sframe.SFrame'> (893580, 6)


In [31]:
usersReqd = 100 #use smaller value during dev, production value should be 10000
uniqueIDs = []
print "len(uniqueIDs) = ", len(uniqueIDs), "\ntype(uniqueIDs) = ", type(uniqueIDs)

counter = 0
uniques = 0
while uniques < usersReqd:
    userID = train_data[counter]['user_id']
    counter += 1
    if userID not in uniqueIDs:
        uniqueIDs.append(userID)
        uniques += 1
        #print "added userID=", userID, " to the set, uniques=", uniques
    #else:
        #print "userID=", userID, " not added to set."
    #print "len(uniqueIDs) = ", len(uniqueIDs)
print "len(uniqueIDs) = ", len(uniqueIDs)

len(uniqueIDs) =  0 
type(uniqueIDs) =  <type 'list'>
len(uniqueIDs) =  100


In [32]:
firstUniques = train_data.filter_by(uniqueIDs, 'user_id')

In [33]:
firstUniques.shape

(1347, 6)

In [36]:
start_time = time.time()
personalized_model = graphlab.item_similarity_recommender.create(firstUniques, user_id='user_id', item_id='song')
print("--- time to graphlab.item_similarity_recommender.create =  %s seconds ---" % (time.time() - start_time))

Recsys training: model = item_similarity

Warning: Ignoring columns song_id, listen_count, title, artist;

To use one of these as a target column, set target = <column_name>

and use a method that allows the use of a target.

Preparing data set.

...

| 7.006ms                             | 100              | 1136            |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.008008s

In [41]:
print "uniqueIDs[0] = ", uniqueIDs[0]
print type(personalized_model)
personalized_model.recommend(users=[uniqueIDs[0]], k=1)
personalized_model.recommend(users=[uniqueIDs[1]], k=1)

uniqueIDs[0] =  b80344d063b5ccb3212f76538f3d9e43d87dca9e
<class 'graphlab.toolkits.recommender.item_similarity_recommender.ItemSimilarityRecommender'>


user_id,song,score,rank
85c1f87fea955d09b4bec2e36aee110927aedf9a ...,Fireflies - CharttraxxKaraoke ...,0.138047616482,1
